<a href="https://colab.research.google.com/github/rajendra1106/stock-price-prediction/blob/main/program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler


In [30]:
from google.colab import files


uploaded = files.upload()


Saving NSE-TATAGLOBAL11.csv to NSE-TATAGLOBAL11.csv


In [56]:
df = pd.read_csv('NSE-TATAGLOBAL11.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index('Date')

In [57]:
train_size = int(len(df) * 0.8)
train_data, test_data = df[:train_size], df[train_size:]

In [58]:
window = 5
ma_predictions = df['Close'].rolling(window=window).mean().iloc[window:].values

In [59]:
regression_model = LinearRegression()
regression_model.fit(pd.Series(range(len(train_data))).values.reshape(-1, 1), train_data['Close'].values.reshape(-1, 1))
lr_predictions = regression_model.predict(pd.Series(range(len(df))).values.reshape(-1, 1)).flatten()

In [60]:
knn_model = KNeighborsRegressor(n_neighbors=5)
knn_model.fit(pd.Series(range(len(train_data))).values.reshape(-1, 1), train_data['Close'].values.reshape(-1, 1))
knn_predictions = knn_model.predict(pd.Series(range(len(df))).values.reshape(-1, 1)).flatten()

In [61]:
arima_model = ARIMA(train_data['Close'].values, order=(1, 1, 1))
arima_model_fit = arima_model.fit()
arima_predictions = arima_model_fit.predict(start=len(train_data), end=len(df)-1, typ='levels')

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [64]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df['Close'].values.reshape(-1, 1))
train_X, train_y = [], []
for i in range(window, len(train_data)):
    train_X.append(scaled_data[i-window:i, 0])
    train_y.append(scaled_data[i, 0])
train_X, train_y = np.array(train_X), np.array(train_y)
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(train_X.shape[1], 1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(units=1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')
lstm_model.fit(train_X, train_y, epochs=50, batch_size=32)

lstm_inputs = scaled_data[len(df)-window:, 0]
lstm_inputs = np.reshape(lstm_inputs, (1, window, 1))
lstm_predictions = lstm_model.predict(lstm_inputs)
lstm_predictions = scaler.inverse_transform(lstm_predictions).flatten()

Epoch 1/50
31/31 [==============================] - 5s 8ms/step - loss: 0.0418
Epoch 2/50
31/31 [==============================] - 0s 8ms/step - loss: 0.0025
Epoch 3/50
31/31 [==============================] - 0s 8ms/step - loss: 9.1968e-04
Epoch 4/50
31/31 [==============================] - 0s 8ms/step - loss: 8.5449e-04
Epoch 5/50
31/31 [==============================] - 0s 9ms/step - loss: 8.3627e-04
Epoch 6/50
31/31 [==============================] - 0s 8ms/step - loss: 8.5369e-04
Epoch 7/50
31/31 [==============================] - 0s 8ms/step - loss: 8.2417e-04
Epoch 8/50
31/31 [==============================] - 0s 8ms/step - loss: 9.1766e-04
Epoch 9/50
31/31 [==============================] - 0s 9ms/step - loss: 8.7564e-04
Epoch 10/50
31/31 [==============================] - 0s 11ms/step - loss: 8.1730e-04
Epoch 11/50
31/31 [==============================] - 0s 13ms/step - loss: 8.2660e-04
Epoch 12/50
31/31 [==============================] - 0s 14ms/step - loss: 8.1566e-04
Epoch 

In [65]:
test_price = test_data['Close'].values
models = ['Moving Average', 'Linear Regression', 'k-Nearest Neighbors', 'Auto ARIMA', 'LSTM']
predictions = [ma_predictions, lr_predictions, knn_predictions, arima_predictions, lstm_predictions]

In [79]:
for model, prediction in zip(models, predictions):
    prediction = prediction[:len(test_price)].flatten()  # Adjust the length and shape of prediction
    test_price = test_price[:len(prediction)]  # Adjust the length of test_price
    mse = mean_squared_error(test_price, prediction)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test_price - prediction) / test_price)) * 100

    # Print or store the evaluation metrics
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"MAPE: {mape}")

MSE: 13236.844455186721
RMSE: 115.05148610594615
MAPE: 73.09772156197106
MSE: 5416.265549583457
RMSE: 73.59528211497974
MAPE: 47.793875192321586
MSE: 13244.422658506224
RMSE: 115.08441535892783
MAPE: 73.16592856888836
MSE: 140.10796186978814
RMSE: 11.836720908671799
MAPE: 6.762349975217726
MSE: 16.045540171265692
RMSE: 4.005688476562511
MAPE: 2.498869916757649


In [80]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate evaluation metrics for each model
evaluation_metrics = []
for model, prediction in zip(models, predictions):
    prediction = prediction[:len(test_price)].flatten()  # Adjust the length and shape of prediction
    mse = mean_squared_error(test_price, prediction)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((test_price - prediction) / test_price)) * 100
    evaluation_metrics.append((mse, rmse, mape))

# Print evaluation metrics for each model
for i, (model, prediction) in enumerate(zip(models, predictions)):
    print(f"Model {i+1} Evaluation Metrics:")
    print("MSE:", evaluation_metrics[i][0])
    print("RMSE:", evaluation_metrics[i][1])
    print("MAPE:", evaluation_metrics[i][2])
    print()

# Compare the models and identify the best performer
best_model_index = np.argmin([metric[0] for metric in evaluation_metrics])
best_model = models[best_model_index]
best_prediction = predictions[best_model_index]

# Make conclusions and recommendations based on the evaluation
print("Best Model Evaluation Metrics:")
print("MSE:", evaluation_metrics[best_model_index][0])
print("RMSE:", evaluation_metrics[best_model_index][1])
print("MAPE:", evaluation_metrics[best_model_index][2])
print()
print("Recommendation: Model", best_model_index+1, "performed the best.")

Model 1 Evaluation Metrics:
MSE: 4048.7768999999994
RMSE: 63.629999999999995
MAPE: 39.6943231441048

Model 2 Evaluation Metrics:
MSE: 6901.691436321516
RMSE: 83.07641925553554
MAPE: 51.82558905523115

Model 3 Evaluation Metrics:
MSE: 3589.208099999996
RMSE: 59.90999999999997
MAPE: 37.3736743605739

Model 4 Evaluation Metrics:
MSE: 0.8873306939400439
RMSE: 0.9419823214583403
MAPE: 0.5876371312902934

Model 5 Evaluation Metrics:
MSE: 16.045540171265692
RMSE: 4.005688476562511
MAPE: 2.498869916757649

Best Model Evaluation Metrics:
MSE: 0.8873306939400439
RMSE: 0.9419823214583403
MAPE: 0.5876371312902934

Recommendation: Model 4 performed the best.
